In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
#Import the Data set
df = pd.read_csv("data\\Traffic.csv")